In [1]:
import gradio as gr
import json
import pickle

/home/betty/miniconda3/lib/python3.12/site-packages/numpy/_core/getlimits.py:548: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
# evaluation_result structure
# evaluation_result ={
#                    "consistency":
#                             {
#                           "scenario_consistency": {
#                             "location": true,
#                             "context": true,
#                             "conversation_context_type": true,
#                             "main_topic": true,
#                             "goal_alignment": true,
#                             "role_nationality": true,
#                             "overall_scenario_consistency_score": 5
#                           },
#                           "self_consistency": {
#                             "role_names": true,
#                             "role_age": true,
#                             "role_occupation": true,
#                             "role_relationship": true,
#                             "time_of_day": true,
#                             "emotional_tone": true,
#                             "relationship_dynamic": true,
#                             "self_introduction": true,
#                             "setting_context_alignment": true,
#                             "overall_self_consistency_score": 5
#                           }
#                         },
#                     "coherence":
#                         [{
#                           "turn_id": <integer>,
#                           "topic_relevance": true,
#                           "contextual_follow_up": true,
#                           "logical_continuity": true,
#                           "no_contradiction": true,
#                           "script_alignment": true,
#                           "coherence_score": 5
#                         }],
#                    "naturalness":
#                     [{
#                         "turn_id": <integer>,
#                         "oral_style": true,
#                         "length_and_flow": true,
#                         "emotion_appropriateness": true,
#                         "text_emotion_consistency": true,
#                         "contextual_vocabulary_style": true,
#                         "naturalness_score": 5
#                     }]}

In [3]:
# question structure
# questions = [
#     {
#         "id": 1,
#         "metadata": metadata,
#         "conversation": {
#             "conversation": [
#                 {
#                     'speaker': 'Ethan',
#                     'text': "Hi, I'm looking for a new smart home security system. Can you show me some options?",
#                     'emotion': 'curious',
#                     'speech_rate': 'medium',
#                     'pause_after': 'short'
#                 },
#                 {
#                     'speaker': 'Alexandra',
#                     'text': "Absolutely! We have a few different systems available. Can you tell me a bit about your current setup and what you're looking for in a new system?",
#                     'emotion': 'helpful',
#                     'speech_rate': 'medium',
#                     'pause_after': 'medium'
#                 }
#             ]
#         },
#         "evaluation_tag": False,
#         "evaluation_result": evaluation_result
#     }
# ]

In [2]:
def add_info_to_pkl(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    for index, item in enumerate(data):
        # Add unique ID and evaluation tag
        item['id'] = index + 1
        item['evaluation_tag'] = False

        # Build evaluation_result
        conversation_length = len(item['conversation']['conversation'])
        evaluation_result = {
            "consistency": {
                "scenario_consistency": {
                    "location": True,
                    "context": True,
                    "conversation_context_type": True,
                    "main_topic": True,
                    "goal_alignment": True,
                    "role_nationality": True,
                    "overall_scenario_consistency_score": 5
                },
                "self_consistency": {
                    "role_names": True,
                    "role_age": True,
                    "role_occupation": True,
                    "role_relationship": True,
                    "time_of_day": True,
                    "emotional_tone": True,
                    "relationship_dynamic": True,
                    "self_introduction": True,
                    "setting_context_alignment": True,
                    "overall_self_consistency_score": 5
                }
            },
            "coherence": [
                {
                    "turn_id": turn_id + 1,
                    "topic_relevance": True,
                    "contextual_follow_up": True,
                    "logical_continuity": True,
                    "no_contradiction": True,
                    "script_alignment": True,
                    "coherence_score": 5
                }
                for turn_id in range(conversation_length)
            ],
            "naturalness": [
                {
                    "turn_id": turn_id + 1,
                    "oral_style": True,
                    "length_and_flow": True,
                    "emotion_appropriateness": True,
                    "text_emotion_consistency": True,
                    "contextual_vocabulary_style": True,
                    "naturalness_score": 5
                }
                for turn_id in range(conversation_length)
            ]
        }
        item['evaluation_result'] = evaluation_result

    with open(file_path, 'wb') as f:
        pickle.dump(data, f)


In [3]:
add_info_to_pkl('dialogues.pkl')

In [4]:
with open('dialogues.pkl', 'rb') as f:
    questions = pickle.load(f)

In [6]:
# 检查加载的数据
# questions[0]
# questions[0]['metadata']
# questions[0]['script']
# questions[0]['conversation']['conversation'][0]
# questions[0]['evaluation_result']

In [5]:
current_index = 0

# Load question metadata
def load_metadata(question_index):
    metadata = questions[question_index]["metadata"]
    return metadata

# Gradio Interface for Questions
def interface():
    global current_index

    # Inputs for each metadata field
    interaction_type = gr.Textbox(label="Interaction Type", interactive=False)
    location = gr.Textbox(label="Location", interactive=False)
    goal = gr.Textbox(label="Goal", interactive=False)
    cultural_background = gr.Textbox(label="Cultural Background", interactive=False)

    setting_location = gr.Textbox(label="Setting Location", interactive=False)
    time_of_day = gr.Textbox(label="Time of Day", interactive=False)
    context = gr.Textbox(label="Context", interactive=False)

    role_1_name = gr.Textbox(label="Role 1 Name", interactive=False)
    role_1_gender = gr.Textbox(label="Role 1 Gender", interactive=False)
    role_1_age = gr.Textbox(label="Role 1 Age", interactive=False)
    role_1_occupation = gr.Textbox(label="Role 1 Occupation", interactive=False)
    role_1_nationality = gr.Textbox(label="Role 1 Nationality", interactive=False)
    role_1_personality_traits = gr.Textbox(label="Role 1 Personality Traits", interactive=False)
    role_1_relationship_context = gr.Textbox(label="Role 1 Relationship Context", interactive=False)
    role_1_self_introduction = gr.Textbox(label="Role 1 Self Introduction", interactive=False, lines=5)

    role_2_name = gr.Textbox(label="Role 2 Name", interactive=False)
    role_2_gender = gr.Textbox(label="Role 2 Gender", interactive=False)
    role_2_age = gr.Textbox(label="Role 2 Age", interactive=False)
    role_2_occupation = gr.Textbox(label="Role 2 Occupation", interactive=False)
    role_2_nationality = gr.Textbox(label="Role 2 Nationality", interactive=False)
    role_2_personality_traits = gr.Textbox(label="Role 2 Personality Traits", interactive=False)
    role_2_relationship_context = gr.Textbox(label="Role 2 Relationship Context", interactive=False)
    role_2_self_introduction = gr.Textbox(label="Role 2 Self Introduction", interactive=False, lines=5)

    conversation_type = gr.Textbox(label="Conversation Type", interactive=False)
    main_topic = gr.Textbox(label="Main Topic", interactive=False)
    relationship_dynamic = gr.Textbox(label="Relationship Dynamic", interactive=False)
    emotional_tone = gr.Textbox(label="Emotional Tone", interactive=False)
    expected_duration = gr.Textbox(label="Expected Duration", interactive=False)
    expected_turns = gr.Textbox(label="Expected Turns", interactive=False)
    key_points = gr.Textbox(label="Key Points", interactive=False, lines=5)

    next_button = gr.Button("Next Question")

    # Update Metadata function
    def update_metadata():
        global current_index
        if current_index < len(questions):
            question = questions[current_index]
            metadata = question["metadata"]
            conversation = question["conversation"]["conversation"]
            question_id = question["id"]
            # Collect the values from the editable fields for consistency
            consistency_values = {
                "scenario_consistency": {},
                "self_consistency": {}
            }

            # Collect scenario consistency values
            for key in questions[0]['evaluation_result']['consistency']['scenario_consistency']:
                consistency_values["scenario_consistency"][key] = globals()[key].value

            # Collect self consistency values
            for key in questions[0]['evaluation_result']['consistency']['self_consistency']:
                consistency_values["self_consistency"][key] = globals()[key].value

            # Save these to the dictionary
            questions[current_index]['evaluation_result']['consistency'] = consistency_values

            current_index += 1
            return (
                question_id,
                metadata['input_scenario']['interaction_type'],
                metadata['input_scenario']['location'],
                metadata['input_scenario']['goal'],
                metadata['input_scenario']['cultural_background'],

                metadata['setting']['location'],
                metadata['setting']['time_of_day'],
                metadata['setting']['context'],

                metadata['role_1']['name'],
                metadata['role_1']['gender'],
                str(metadata['role_1']['age']),
                metadata['role_1']['occupation'],
                metadata['role_1']['nationality'],
                ", ".join(metadata['role_1']['personality_traits']),
                metadata['role_1']['relationship_context'],
                metadata['role_1']['self_introduction'],

                metadata['role_2']['name'],
                metadata['role_2']['gender'],
                str(metadata['role_2']['age']),
                metadata['role_2']['occupation'],
                metadata['role_2']['nationality'],
                ", ".join(metadata['role_2']['personality_traits']),
                metadata['role_2']['relationship_context'],
                metadata['role_2']['self_introduction'],

                metadata['conversation_context']['type'],
                metadata['conversation_context']['main_topic'],
                metadata['conversation_context']['relationship_dynamic'],
                metadata['conversation_context']['emotional_tone'],
                metadata['conversation_context']['expected_duration'],
                str(metadata['conversation_context']['expected_turns']),
                "\n".join(metadata['conversation_context']['key_points']),
                conversation
            )
        else:
            return ("",) * 32

    with gr.Blocks() as app:
        gr.Markdown("# Evaluation Interface")
        gr.Textbox(value="ID: " + str(questions[0]["id"]), label="Question ID", interactive=False)
        gr.Markdown("# Questionnaire Metadata Viewer")

        with gr.Tab("Input Scenario"):
            with gr.Row():
                interaction_type.render()
                location.render()
            with gr.Row():
                goal.render()
                cultural_background.render()

        with gr.Tab("Setting"):
            with gr.Row():
                setting_location.render()
                time_of_day.render()
            with gr.Row():
                context.render()

        with gr.Tab("Role 1"):
            with gr.Row():
                role_1_name.render()
                role_1_gender.render()
            with gr.Row():
                role_1_age.render()
                role_1_occupation.render()
            with gr.Row():
                role_1_nationality.render()
                role_1_personality_traits.render()
            with gr.Row():
                role_1_relationship_context.render()
            role_1_self_introduction.render()

        with gr.Tab("Role 2"):
            with gr.Row():
                role_2_name.render()
                role_2_gender.render()
            with gr.Row():
                role_2_age.render()
                role_2_occupation.render()
            with gr.Row():
                role_2_nationality.render()
                role_2_personality_traits.render()
            with gr.Row():
                role_2_relationship_context.render()
            role_2_self_introduction.render()

        with gr.Tab("Conversation Context"):
            with gr.Row():
                conversation_type.render()
                main_topic.render()
            with gr.Row():
                relationship_dynamic.render()
                emotional_tone.render()
            with gr.Row():
                expected_duration.render()
                expected_turns.render()
            key_points.render()

        gr.Markdown("# Consistency")
        with gr.Accordion("Scenario Consistency"):
            with gr.Row():
                for key, value in questions[0]['evaluation_result']['consistency']['scenario_consistency'].items():
                    gr.Textbox(value=key, label="Key", interactive=False)
                    gr.Textbox(value=value, label="Value", interactive=True)

        with gr.Accordion("Self Consistency"):
            with gr.Row():
                for key, value in questions[0]['evaluation_result']['consistency']['self_consistency'].items():
                    gr.Textbox(value=key, label="Key", interactive=False)
                    gr.Textbox(value=value, label="Value", interactive=True)

        gr.Markdown("# Conversation Log")
        with gr.Accordion("Conversation Details"):
            for turn in questions[0]['conversation']['conversation']:
                with gr.Accordion(f"{turn['speaker']}"):
                    gr.Textbox(value=f"{turn['text']}", interactive=False, lines=2, label="Text")
                    gr.Textbox(value=f"Emotion: {turn['emotion']}, Speech Rate: {turn['speech_rate']}, Pause After: {turn['pause_after']}", interactive=False, label="Details")

        next_button.render()

        # Click Next Button to Update Metadata
        next_button.click(
            fn=update_metadata,
            inputs=[],
            outputs=[
                gr.Textbox(),
                interaction_type,
                location,
                goal,
                cultural_background,
                
                setting_location,
                time_of_day,
                context,
                
                role_1_name,
                role_1_gender,
                role_1_age,
                role_1_occupation,
                role_1_nationality,
                role_1_personality_traits,
                role_1_relationship_context,
                role_1_self_introduction,

                role_2_name,
                role_2_gender,
                role_2_age,
                role_2_occupation,
                role_2_nationality,
                role_2_personality_traits,
                role_2_relationship_context,
                role_2_self_introduction,

                conversation_type,
                main_topic,
                relationship_dynamic,
                emotional_tone,
                expected_duration,
                expected_turns,
                key_points,
            ]
        )

    return app

# Launch Gradio App
if __name__ == "__main__":
    app = interface()
    app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [5]:
current_index = 0

# Load question metadata
def load_metadata(question_index):
    metadata = questions[question_index]["metadata"]
    return metadata

# Gradio Interface for Questions
def interface():
    global current_index

    # Inputs for each metadata field
    interaction_type = gr.Textbox(label="Interaction Type", interactive=False)
    location = gr.Textbox(label="Location", interactive=False)
    goal = gr.Textbox(label="Goal", interactive=False)
    cultural_background = gr.Textbox(label="Cultural Background", interactive=False)

    setting_location = gr.Textbox(label="Setting Location", interactive=False)
    time_of_day = gr.Textbox(label="Time of Day", interactive=False)
    context = gr.Textbox(label="Context", interactive=False)

    role_1_name = gr.Textbox(label="Role 1 Name", interactive=False)
    role_1_gender = gr.Textbox(label="Role 1 Gender", interactive=False)
    role_1_age = gr.Textbox(label="Role 1 Age", interactive=False)
    role_1_occupation = gr.Textbox(label="Role 1 Occupation", interactive=False)
    role_1_nationality = gr.Textbox(label="Role 1 Nationality", interactive=False)
    role_1_personality_traits = gr.Textbox(label="Role 1 Personality Traits", interactive=False)
    role_1_relationship_context = gr.Textbox(label="Role 1 Relationship Context", interactive=False)
    role_1_self_introduction = gr.Textbox(label="Role 1 Self Introduction", interactive=False, lines=5)

    role_2_name = gr.Textbox(label="Role 2 Name", interactive=False)
    role_2_gender = gr.Textbox(label="Role 2 Gender", interactive=False)
    role_2_age = gr.Textbox(label="Role 2 Age", interactive=False)
    role_2_occupation = gr.Textbox(label="Role 2 Occupation", interactive=False)
    role_2_nationality = gr.Textbox(label="Role 2 Nationality", interactive=False)
    role_2_personality_traits = gr.Textbox(label="Role 2 Personality Traits", interactive=False)
    role_2_relationship_context = gr.Textbox(label="Role 2 Relationship Context", interactive=False)
    role_2_self_introduction = gr.Textbox(label="Role 2 Self Introduction", interactive=False, lines=5)

    conversation_type = gr.Textbox(label="Conversation Type", interactive=False)
    main_topic = gr.Textbox(label="Main Topic", interactive=False)
    relationship_dynamic = gr.Textbox(label="Relationship Dynamic", interactive=False)
    emotional_tone = gr.Textbox(label="Emotional Tone", interactive=False)
    expected_duration = gr.Textbox(label="Expected Duration", interactive=False)
    expected_turns = gr.Textbox(label="Expected Turns", interactive=False)
    key_points = gr.Textbox(label="Key Points", interactive=False, lines=5)

    next_button = gr.Button("Next Question")

    # Update Metadata function
    def update_metadata():
        global current_index
        if current_index < len(questions):
            question = questions[current_index]
            metadata = question["metadata"]
            conversation = question["conversation"]["conversation"]
            question_id = question["id"]
            # Collect the values from the editable fields for consistency
            consistency_values = {
                "scenario_consistency": {},
                "self_consistency": {}
            }

            # Collect scenario consistency values
            for key in questions[0]['evaluation_result']['consistency']['scenario_consistency']:
                consistency_values["scenario_consistency"][key] = globals()[key].value

            # Collect self consistency values
            for key in questions[0]['evaluation_result']['consistency']['self_consistency']:
                consistency_values["self_consistency"][key] = globals()[key].value

            # Save these to the dictionary
            questions[current_index]['evaluation_result']['consistency'] = consistency_values

            current_index += 1
            return (
                question_id,
                metadata['input_scenario']['interaction_type'],
                metadata['input_scenario']['location'],
                metadata['input_scenario']['goal'],
                metadata['input_scenario']['cultural_background'],

                metadata['setting']['location'],
                metadata['setting']['time_of_day'],
                metadata['setting']['context'],

                metadata['role_1']['name'],
                metadata['role_1']['gender'],
                str(metadata['role_1']['age']),
                metadata['role_1']['occupation'],
                metadata['role_1']['nationality'],
                ", ".join(metadata['role_1']['personality_traits']),
                metadata['role_1']['relationship_context'],
                metadata['role_1']['self_introduction'],

                metadata['role_2']['name'],
                metadata['role_2']['gender'],
                str(metadata['role_2']['age']),
                metadata['role_2']['occupation'],
                metadata['role_2']['nationality'],
                ", ".join(metadata['role_2']['personality_traits']),
                metadata['role_2']['relationship_context'],
                metadata['role_2']['self_introduction'],

                metadata['conversation_context']['type'],
                metadata['conversation_context']['main_topic'],
                metadata['conversation_context']['relationship_dynamic'],
                metadata['conversation_context']['emotional_tone'],
                metadata['conversation_context']['expected_duration'],
                str(metadata['conversation_context']['expected_turns']),
                "\n".join(metadata['conversation_context']['key_points']),
                conversation
            )
        else:
            return ("",) * 32

    with gr.Blocks() as app:
        gr.Markdown("# Evaluation Interface")
        gr.Textbox(value="ID: " + str(questions[0]["id"]), label="Question ID", interactive=False)
        gr.Markdown("# Questionnaire Metadata Viewer")

        with gr.Tab("Input Scenario"):
            with gr.Row():
                interaction_type.render()
                location.render()
            with gr.Row():
                goal.render()
                cultural_background.render()

        with gr.Tab("Setting"):
            with gr.Row():
                setting_location.render()
                time_of_day.render()
            with gr.Row():
                context.render()

        with gr.Tab("Role 1"):
            with gr.Row():
                role_1_name.render()
                role_1_gender.render()
            with gr.Row():
                role_1_age.render()
                role_1_occupation.render()
            with gr.Row():
                role_1_nationality.render()
                role_1_personality_traits.render()
            with gr.Row():
                role_1_relationship_context.render()
            role_1_self_introduction.render()

        with gr.Tab("Role 2"):
            with gr.Row():
                role_2_name.render()
                role_2_gender.render()
            with gr.Row():
                role_2_age.render()
                role_2_occupation.render()
            with gr.Row():
                role_2_nationality.render()
                role_2_personality_traits.render()
            with gr.Row():
                role_2_relationship_context.render()
            role_2_self_introduction.render()

        with gr.Tab("Conversation Context"):
            with gr.Row():
                conversation_type.render()
                main_topic.render()
            with gr.Row():
                relationship_dynamic.render()
                emotional_tone.render()
            with gr.Row():
                expected_duration.render()
                expected_turns.render()
            key_points.render()

        gr.Markdown("# Consistency")
        with gr.Accordion("Scenario Consistency"):
            with gr.Row():
                for key, value in questions[0]['evaluation_result']['consistency']['scenario_consistency'].items():
                    # gr.Textbox(value=key, label="Key", interactive=False)
                    gr.Textbox(value=value, label=key, interactive=True)

        with gr.Accordion("Self Consistency"):
            with gr.Row():
                for key, value in questions[0]['evaluation_result']['consistency']['self_consistency'].items():
                    # gr.Textbox(value=key, label="Key", interactive=False)
                    gr.Textbox(value=value, label=key, interactive=True)

        gr.Markdown("# Conversation Log")
        with gr.Accordion("Conversation Details"):
            for turn in questions[0]['conversation']['conversation']:
                with gr.Accordion(f"{turn['speaker']}"):
                    gr.Textbox(value=f"{turn['text']}", interactive=False, lines=2, label="Text")
                    gr.Textbox(value=f"Emotion: {turn['emotion']}, Speech Rate: {turn['speech_rate']}, Pause After: {turn['pause_after']}", interactive=False, label="Details")

        next_button.render()

        # Click Next Button to Update Metadata
        next_button.click(
            fn=update_metadata,
            inputs=[],
            outputs=[
                gr.Textbox(),
                interaction_type,
                location,
                goal,
                cultural_background,
                
                setting_location,
                time_of_day,
                context,
                
                role_1_name,
                role_1_gender,
                role_1_age,
                role_1_occupation,
                role_1_nationality,
                role_1_personality_traits,
                role_1_relationship_context,
                role_1_self_introduction,

                role_2_name,
                role_2_gender,
                role_2_age,
                role_2_occupation,
                role_2_nationality,
                role_2_personality_traits,
                role_2_relationship_context,
                role_2_self_introduction,

                conversation_type,
                main_topic,
                relationship_dynamic,
                emotional_tone,
                expected_duration,
                expected_turns,
                key_points,
            ]
        )

    return app

# Launch Gradio App
if __name__ == "__main__":
    app = interface()
    app.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
